In [5]:
import pandas as pd
import nfl_data_py as nfl
from datetime import datetime
from dateutil import relativedelta

In [37]:
def get_data(year: int):
    if year < 2006:
        raise Exception('Date must be greater than 2005')
    data = nfl.import_weekly_data([year])
    data = data[data.season_type == 'REG']
    names = data[['player_id', 'player_name', 'season']]
    names = names.drop_duplicates(subset='player_id')
    data = data.drop(columns=['week', 'season', 'rushing_2pt_conversions', 'receptions', 'targets', 'receiving_yards', 'receiving_tds', 'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
                              'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa', 'receiving_2pt_conversions', 'special_teams_tds', 'fantasy_points_ppr', 'racr', 'target_share', 'air_yards_share', 'wopr'])
    games = data.groupby('player_id').count()
    games['games'] = games['player_name']
    games = games['games']
    grouped = data.groupby('player_id').sum()

    qb_data = names.join(grouped, on='player_id', how='inner')
    qb_data = qb_data.join(games, on='player_id', how='inner')

    id_data = nfl.import_ids(ids=['gsis', 'espn'])
    id_data = id_data[['name', 'position', 'birthdate', 'height', 'weight', 'college', 'draft_year', 'draft_round', 'draft_pick', 'gsis_id', 'espn_id']]
    id_data = id_data[id_data.position=='QB']
    qb_data = qb_data.merge(id_data, how='inner', left_on='player_id', right_on='gsis_id')

    ages = [relativedelta.relativedelta(datetime.strptime(str(row[3]) + '-12-31', '%Y-%m-%d').date(), datetime.strptime(row[31], '%Y-%m-%d').date()).years for row in qb_data.itertuples()]
    qb_data['age'] = pd.Series(ages)

    qbrs = nfl.import_qbr([year])
    qbrs = qbrs[qbrs.season_type == 'Regular']
    qbrs = qbrs[['name_first', 'name_last', 'headshot_href', 'rank', 'qbr_total', 'pts_added', 'qb_plays', 'epa_total', 'pass', 'run', 'exp_sack', 'penalty', 'qbr_raw', 'team', 'qualified', 'player_id']]
    qb_data = qb_data.merge(qbrs, how='inner', left_on='espn_id', right_on='player_id')

    qb_data = qb_data[['name', 'position', 'age', 'season', 'games', 'team', 'rank', 'completions', 'attempts', 'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards', 'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards', 'passing_yards_after_catch',
                       'passing_first_downs', 'passing_epa', 'passing_2pt_conversions', 'dakota', 'carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_epa', 'fantasy_points', 'qbr_total', 'pts_added', 'qb_plays',
                       'epa_total', 'pass', 'run', 'qbr_raw', 'qualified', 'height', 'weight', 'name_first', 'name_last', 'draft_round', 'draft_pick', 'gsis_id', 'espn_id', 'headshot_href']]

    return qb_data

In [62]:
def combine_data(years: list):
    cols = ['espn_id', 'games', 'rank', 'completions', 'attempts', 'passing_yards', 'passing_tds', 'interceptions', 'sacks',
            'sack_yards', 'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards', 'passing_yards_after_catch',
            'passing_first_downs', 'passing_epa', 'passing_2pt_conversions', 'dakota', 'carries', 'rushing_yards',
            'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_epa',
            'fantasy_points', 'qbr_total', 'pts_added', 'qb_plays', 'epa_total', 'pass', 'run', 'qbr_raw', 'qualified']
    data = [get_data(year) for year in years]

    for i in reversed(range(2, len(data))):
        data[i] = data[i].merge(data[i-1][cols],
                                left_on='espn_id',
                                right_on='espn_id',
                                how='left',
                                suffixes=('', '_n-1'))
        data[i] = data[i].merge(data[i-2][cols],
                                left_on='espn_id',
                                right_on='espn_id',
                                how='left',
                                suffixes=('', '_n-2'))
    for i in range(len(data) - 1):
        data[i] = data[i].merge(data[i + 1][['fantasy_points', 'espn_id']],
                                  left_on='espn_id',
                                  right_on='espn_id',
                                  how='left',
                                  suffixes=('', '_next_year'))
    qb_data = pd.DataFrame()
    for dataframe in data:
        qb_data = pd.concat([qb_data, dataframe],
                            ignore_index=True)

    return qb_data

In [63]:
%%time
data = combine_data([year for year in range(2006, 2022)])

Downcasting floats.


/Users/benabraham/opt/anaconda3/envs/FantFootball/lib/python3.9/site-packages/nfl_data_py/__init__.py:637: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df = df[set(tgt_ids + rem_cols)]


Downcasting floats.


/Users/benabraham/opt/anaconda3/envs/FantFootball/lib/python3.9/site-packages/nfl_data_py/__init__.py:637: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df = df[set(tgt_ids + rem_cols)]


Downcasting floats.


...

Downcasting floats.


/Users/benabraham/opt/anaconda3/envs/FantFootball/lib/python3.9/site-packages/nfl_data_py/__init__.py:637: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df = df[set(tgt_ids + rem_cols)]


Downcasting floats.


/Users/benabraham/opt/anaconda3/envs/FantFootball/lib/python3.9/site-packages/nfl_data_py/__init__.py:637: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df = df[set(tgt_ids + rem_cols)]


CPU times: user 4.1 s, sys: 1.09 s, total: 5.19 s
Wall time: 45.3 s


In [78]:
data.to_parquet('qb_season_data.parquet')